### 04 - Libraries

#### Outline

* Performance: clocks, timeit, cProfile, & snakeviz
* Numerics: numpy & scipy
* Plotting: matplotlib & plotly
* Autodiff: Jax, Casadi, and Aerosandbox
* Symbolics
* Fluid Properties

#### Commentary

There are tons of fantastic libraries for python! The ones listed here are just a selection of essentials.

Later in the class, we'll examine more specialized ones for numerical methods, optimization, etc.

----
#### Performance

Most of the time, you don't need to worry about performance. Computers are very, very fast.

That said, especially for numerical work, we often run into problems where if we don't make time for performance, it will make time for itself.

The key thing is to **_make sure you are optimizing for a reason_** - if you'll only run a script once and it's fast enough, time spent making it faster **may be wasteful**.

Some patterns to look out for, and what to do instead:

| Pattern | Alternative |
|---------|-------------|
| Looping over arrays to do math one element at a time in python | Use array broadcasting |
| Nested for-loops to handle 2D+ data | Flatten & reshape if the calc supports it |
| Factorial recombination of cases | Monte Carlo or optimization |

There are, of course, plenty of other ways to find low-performing code and optimize out from under it.

Python isn't a common target for high-effort performance optimization - usually, the pattern is to profile Python code, find the expensive bits, and figure out how to get those to run in a compiled language instead (usually using numpy, which is written in C). Tools for compiled languages, like disassembly, valgrind/cachegrind, strace, perf, and so on, find minimal utility in examining Python scripts.

To that end, the main tools are:
* time.monotonic_ns - direct-ish access to CPU perf counter (monotonic clock). Just check how long things take!
* timeit.timeit - simple benchmarking utility
* cProfile & snakeviz - flame charts of profile results for finding what's really important
* pyinstrument - ergonomic call-stack profiler/visualizer combining most of cProfile+snakeviz

All of these tools are for **_measuring performance_** - if you take one thing away from this, it should be that when you do optimize code, it should be **based on real evidence and measurements** that indicate that the effort will be of value.

Don't spend effort guessing what's slow and optimizing the wrong thing when you can just check!

In [ ]:
# ruff: noqa: E731
# ^ disables lint on assigning lambda
import os
from pathlib import Path

import numpy as np
from time import monotonic_ns

from timeit import timeit

import matplotlib.pyplot as plt

working_directory = Path(
    os.path.abspath("")
)  # Immediately stuff the string into a Path object
static_dir = working_directory / "static" / "04"

In [ ]:
# Some test data
a = np.random.uniform(-1.0, 1.0, 100)
b = np.random.uniform(-1.0, 1.0, 100)

##### Monotonics

`time.perf_counter_ns` is a simple interface to a monotonic clock that is fast to access and never runs backward, also called a "perf counter."

Normal date/time clocks ("realtime") can run backward briefly and may be slow to access.

In [ ]:
# Run this a few times and look at the noise! Different number every time
start_ns = monotonic_ns()
np.outer(a, b)
end_ns = monotonic_ns()

dt = (end_ns - start_ns) / 1e9
print(f"Finished in {dt:2e} sec")

##### Timeit

In [ ]:
%%timeit

# This is some convenient jupyter notebook "magic" for running timeit to benchmark the contents of a cell

a * b

In [ ]:
# If we want to see how things change, we can use timeit directly,
# which lets us see the regime transitions from scalar to vector to overrun

n = 10000  # Number of benchmarks

sizes = np.logspace(1, 6, 10)
times = []

for size in sizes:
    # Careful not to include initialization in the benchmark!
    a = np.random.uniform(-1.0, 1.0, int(size))
    b = np.random.uniform(-1.0, 1.0, int(size))

    t = timeit(lambda: a * b, number=n) / n  # Do the benchmark
    # We're only doing this a few times, so it doesn't matter that it's slow
    times.append(t)

fig, axes = plt.subplots(2, 1, sharex=True)  # NOTE: Example of multiple frame layout
plt.sca(axes[0])
plt.plot(sizes, times, color="k")
plt.ylabel("Time [s]")

plt.gca().set_yscale("log")  # NOTE: Example of log-scale axes

plt.sca(axes[1])
plt.plot(sizes, sizes / np.array(times), color="k")
plt.ylabel("Throughput [elems/s]")

# plt.gca().set_yscale("log")
plt.gca().set_xscale("log")

plt.xlabel("# Elems")


##### cProfile & snakeviz

When your script has more than a few lines in it, it may not be obvious where the pain points are. Profile your code to find out!

**Note**: Run snakeviz here as example, since it doesn't embed well. Take bets on what are the slow parts

In [ ]:
import subprocess
import os
from pathlib import Path

here = Path(os.path.abspath(""))
script = here / "script_to_profile.py"
out = here / "script_to_profile.prof"

# Instrument and profile the script
subprocess.check_output(["python", "-m", "cProfile", "-o", str(out), str(script)])

# Make a flame chart of the profile - this is how we'd do it programatically
# subprocess.call(["snakeviz", str(out)])  # This doesn't return

# ..Or in the terminal
# snakeviz ./script_to_profile.prof

# Or, supposedly, in a notebook, but I haven't got this to work
# %load_ext snakeviz # Jupyter "magic" to load a plugin
# %snakeviz "./script_to_profile.prof"

##### Pyinstrument

Pyinstrument combines a profiler and visualizer, and has a jupyter plugin as well as HTML/browser output.

In [ ]:
# Load jupyter plugin
%load_ext pyinstrument

In [ ]:
%%pyinstrument
# Using the jupyter plugin here, but we could also run it
# from the terminal like `pyinstrument -r html ./script_to_profile.prof`
# NOTE: discuss difference in outputs between these two strategies
# (terminal approach captures imports)

from script_to_profile import do_stuff

do_stuff()

----
#### Numerics

`numpy` and `scipy` are the core offering of Python for engineering. They provide 

* Matrices and linear solvers
* Array and matrix construction utilities (linspace, etc)
* Interpolation
* Signal filtering
* Optimizers and rootfinders
* ODE integrators

... and so on.

The `findiff` library also provides easy and reasonably-performant construction of finite-difference operators.

Meanwhile, `sympy` provides symbolic math. For quasi-symbolic differentiation, see the section on automatic differentiation.

##### Arrays, Matrices, and Tensors

**Arrays** are a series of values stored together on disk.<br>
Dense matrices are 2D arrays. Arrays can have more than 2 dimensions.<br>
Tensors are any N-dimensional arrays on which tensor operations are performed.
<br><br>
Arrays' underlying storage can be either `C-contiguous` or `F-contiguous`. Only `C-contiguous` format is in common use and will be discussed here.
* `C-contiguous`: Shape (20, 3) means you have 20 blocks of 3-element-long arrays. `arr[i, :]` refers to a solid block of memory.
* `F-contiguous`: Shape (20, 3) means you have 3 blocks of 20-element-long arrays. `arr[:, i]` refers to a solid block of memory.

This difference in storage is closely related to the difference between columnar and interleaved data; it represents the difference between storing points like `([x0, ..., xn], [y0, ..., yn])` (columnar) vs. `([x0, y0], ..., [xn, yn])` (interleaved).

Slicing an array like `arr[:, i]` produces an **array view** that represents the result *conceptually* but may not be evaluated yet in order to avoid excess computation. Numpy arrays can be either a contiguous array or a discontiguous array view **without external indication**.

Arrays can be reshaped at zero cost by simply changing the interpretation of the underlying dimensions. However, transposing a contiguous array requires a change in storage ordering, and will usually produce an array view that avoids the cost for as long as possible.

**Sparse matrices** are a representation of only the nonzero entries in a matrix.<br>
There are different formats of sparse matrix that have different advantages:

* Triplet (aka COO): easy to construct, but bulky & slow to do math
* Compressed Sparse Column (CSC): easy to decompose / prep solvers; good RHS of matrix-matrix product (fast column access)
* Compressed Sparse Row (CSR): good LHS of matrix-matrix or matrix-vector product (fast row access)

*Triplet format* is simply a series of `[(row, col, val), ...]` 3-element tuples, canonically *sorted by row then by column* within each row to facilitate conversion to other formats. It's the first thing that one would think of if asked to only store nonzero entries.

*Compressed sparse matrix formats* with `nnz` nonzeroes and `n` rows or columns on the compressed axis store
* (size nnz) Array of values 
* (size nnz) Array of uncompressed-axis indices 
* (size n)   Array of pointers to start of each row or column in the other arrays (whichever is compressed)

The transpose of a compressed sparse matrix can be done at zero cost by simply changing the name from CSR to CSC or CSC to CSR, because the significance of the rows and columns can be swapped directly.

##### Numpy - Arrays and Matrices

In [ ]:
# Numpy provides methods for constructing and manipulating dense arrays and matrices.

# Building monotonic series of values (for plotting, etc)
a = np.linspace(0.0, 10.0, 25)
b = np.arange(0.0, 10.0, 0.04)

# Copy an array to prevent unexpected mutation
a_sliced = a[::2].copy()

# Make an array contiguous, copying only if necessary
a_sliced_contig = np.ascontiguousarray(a[::2])  # This will copy
a_contig = np.ascontiguousarray(a)  # This will return a reference

# Building 2D meshes of values (for exploring a space, plotting, etc)
X, Y = np.meshgrid(a, b, indexing="ij")  # "ij" indexing makes the intuitive result

# Generating random numbers
mean, std = (0.0, 1.414)
c = np.random.normal(loc=mean, scale=std, size=100)

# Flattening and reshaping are compatible inverse operations
# They are zero-cost - the underlying storage does not change, only the interpretation of it!
c2d = c.reshape((10, 10))  # Follows C-ordering
cflat = c2d.flatten()
assert np.all(c == cflat)
# 2D
# [ 1, 0, 0 ]
# [ 0, 1, 0 ]
# [ 0, 0, 1 ]

# Flattened (C-order) is a series of contiguous rows
# [ 1, 0, 0 ][ 0, 1, 0 ][ 0, 0, 1 ]

# Transpose
c2dt = c2d.transpose()
c2dt2 = c2d.T  # Shorthand

# Extract diagonal
cdiag = np.diag(c2d)  # Extract diagonal from matrix
cdiag2 = np.diag(c)  # Create diagonal matrix

# Create block-matrices
eye = np.eye(*c2d.shape)
# fmt: off
cblock = np.block([
    [eye, c2d], 
    [c2d, eye]
])
# fmt: on
print(f"Block-matrix shape: {cblock.shape}")

# Broadcasting operations
c2d * np.ones((10, 1))  # Ok to do (10x10 * 10x1) along first axis
c2d * np.ones((1, 10))  # Ok to do (10x10 * 1x10) along second axis
c2d * np.ones((1, 1))  # Ok to do array-scalar with expanded dimensions
c2d * 1.0  # Ok to do array-scalar with no dimensions

c3d = c.reshape((4, 5, 5))
c3dscaled = c3d * (0.5 * np.ones((4, 1, 1)))  # Multiply each 5x5 sub-array by 0.5
c3dscaled = c3d * (0.5 * np.ones((4, 1, 5)))  # Broadcast along middle axis

try:
    c2d * c  # Not ok to do 10x10 * 100 - no matching axes
except Exception as e:
    print(f"Noop: {e}")


In [ ]:
# In general, numpy is significantly faster to do math operations than python, but only if you use it properly!

npmul = lambda a, b: a * b  # Just use broadcasting!
pymul_comprehension = lambda a, b: np.array([x * y for x, y in zip(a, b)])


def pymul_index(a, b):
    """Don't do stuff like this"""
    out = np.zeros((len(a),))  # Preallocating should help, right?
    for i in range(len(a)):
        out[i] = a[i] * b[i]

    return np.array(out)


def append_loop(a, b):
    """Don't do stuff like this"""
    out = []
    for i in range(len(a)):
        out.append(a[i] * b[i])

    return np.array(out)


n = 100000
t1 = timeit(lambda: pymul_comprehension(c, c), number=n) / n
t1a = timeit(lambda: pymul_index(c, c), number=n) / n
t2 = timeit(lambda: npmul(c, c), number=n) / n
t3 = timeit(lambda: append_loop(c, c), number=n) / n

print(f"Python zip:   {t1:.1e} [s]")
print(f"Python index: {t1a:.1e} [s]")
print(f"Numpy:        {t2:.1e} [s]")
print(f"Append loop:  {t3:.1e} [s]")

print("\nTime ratios compared to numpy")
print(f"Python zip:   {t1 / t2:.1f}")
print(f"Python index: {t1a / t2:.1f}")
print(f"Append loop:  {t3 / t2:.1f}")

In [ ]:
# Operating on discontiguous access patterns in numpy is only slightly slower.
#
# This distinction is much sharper in low-level compiled code, where it can make a difference of a factor of 10-100.
# numpy doesn't mind that much because it's leaving a lot of potential performance on the table by using iterators for everything.

cube = np.random.uniform(0.0, 1.0, 100**3).reshape((100, 100, 100))  # 20x20x20
square = np.random.uniform(0.0, 1.0, 100**2).reshape((100, 100))  # 20x20

contig = lambda a, b: a[2, :, :] * b  # Contiguous access pattern
discontig = lambda a, b: a[:, 2, :] * b  # Scattered access pattern

n = 100000
t1 = timeit(lambda: np.sum(contig(cube, square)), number=n) / n
t2 = timeit(lambda: np.sum(discontig(cube, square)), number=n) / n

print(t2 / t1)


In [ ]:
# A quick warning about `numpy.vectorize`: it doesn't do anything for performance.
# This does not "vectorize" anything!
#
# NOTE: Aside about vectorization as the combined effect of both sequential reads, SIMD, ILP, and cache efficiency
from math import sinh

vsinh = np.vectorize(sinh)

n = 100000
t1 = timeit(lambda: np.array([sinh(x) for x in c]), number=n) / n
t2 = timeit(lambda: vsinh(c), number=n) / n

print(f"Python:            {t1:.2e} [s]")
print(f'Numpy "vectorize": {t2:.2e} [s]')

##### Scipy - Variety Pack

##### Interpolation

In [ ]:
# Interpolation in 1D, 2D, ND
from scipy.interpolate import (
    RegularGridInterpolator,
    RectBivariateSpline,
    interp1d,
    pchip,
    RBFInterpolator,
)
from interpn.multilinear_rectilinear import MultilinearRectilinear

#    Grids
xgrid = np.linspace(0.0, 5.0, 10)
ygrid = np.linspace(-1.0, 1.0, 9)
zgrid = np.linspace(2.0, 3.0, 7)
#    Rectangular meshes
xmesh, ymesh, zmesh = np.meshgrid(xgrid, ygrid, zgrid, indexing="ij")
#    Values at each mesh location
u = np.sin((xmesh**2 + ymesh**2 + zmesh**2) ** 0.5)

#    3D interpolator
interp3d = RegularGridInterpolator((xgrid, ygrid, zgrid), u, method="linear")
interp3d_fast = MultilinearRectilinear.new(
    [xgrid, ygrid, zgrid], u
)  # Faster method for 3D+

#    2D interpolators
interp2d = RegularGridInterpolator((xgrid, ygrid), u[:, :, 0], method="cubic")
interp2d_fast = RectBivariateSpline(
    xgrid, ygrid, u[:, :, 0]
)  # Faster cubic method for 2D only

#    1D interpolators
interp1d_slow = interp1d(xgrid, u[:, 0, 0], kind="linear")
interp1d_fast = lambda x: np.interp(
    x, xgrid, u[:, 0, 0]
)  # Faster method for 1D linear only

#    Specialty interpolators
#        C1-continuous cubic w/ monotonicity constraint
monotonic_hermite_cubic = pchip(xgrid, u[:, 0, 0])
#        Radial basis function can handle point clouds.
#
#        A good rule of thumb is to use a linear kernel with a number of neighbors that reflects
#        the vertices of a rectilinear unit cell in N dimensions (2 for linear, 4 for 2D, 8 for 3D, and so on)
#        which recovers something similar to linear interpolation if the point cloud happens to land on a grid.
point_cloud_x = np.random.uniform(-1.0, 1.0, 10)
point_cloud_y = np.random.uniform(3.14, np.e, 10)
point_cloud_u = np.sin((point_cloud_x**2 + point_cloud_y**2) ** 0.5)
linear_radial_basis = RBFInterpolator(
    np.array((point_cloud_x, point_cloud_y)).T,
    point_cloud_u,
    neighbors=4,
    kernel="linear",
)

##### Root-finding and minimization

Root-finding is a popular method for solving arbitrary systems of equations by rephrasing

$$ f(x) = g(x) $$

as

$$ f_{i}(x_{i}) - g_{i}(x_{i}) = err_{i} $$

and solving to drive the error to zero. This is the underlying scheme behind nearly all methods for solving nonlinear systems, although the method for driving the error to zero varies widely.

In [ ]:
# Nonlinear root-finding and optimization
from scipy.optimize import fsolve, minimize


def f_colebrook(reynolds_number, roughness):
    """
    Implicit Colebrook formula for friction factor. Once upon a time, this was the gold standard
    but newer correlations like Bellos show better experimental agreement and are explicit

    Arguments:
        Re: Reynolds number, with hydraulic diameter as the length scale [units: dimensionless].
        roughness: Pipe inside surface roughness scale height divided by pipe diameter [units: dimensionless].

    Returns:
        Darcy friction factor [units: dimensionless].
    """
    re = reynolds_number
    lhs = lambda f: f**-0.5
    rhs = lambda f: -2 * np.log10((roughness / 3.7) + (2.51 / (re * f**0.5)))

    # Root-finding method
    err = lambda f: np.abs(lhs(f) - rhs(f))
    f = fsolve(err, x0=[0.01], maxfev=1000)[0]  # Root-finder

    # Error minimization method (we happen to know that the minimum of the squared error is zero)
    # err2 = lambda f: (lhs(f) - rhs(f))**2  # Smooth error better for minimizer
    # f = minimize(err2, x0=[0.01], tol=1e-8, method='Nelder-Mead').x[0]  # More general minimizer

    return float(f)


# Run the function to get the friction factor for a given Re and epsilon
f_colebrook(reynolds_number=1e4, roughness=1e-3)

In [ ]:
# Watching the function evaluations of a rootfinder
import matplotlib.pyplot as plt
from numpy.typing import NDArray


def func(x: NDArray, point_log: list[NDArray] | None = None) -> NDArray:
    """Some convex function with logging"""
    if point_log is not None:
        point_log.append(x)  # Telemetry
    out = np.zeros_like(x)
    x0, x1 = x
    out[0] = (x0 / 5 + 1) ** 4 * x1 / 5
    out[1] = np.log((x1 / 5) ** 2 + 1.0) * (x0 / 5) ** 2
    return out


x0 = np.array([14.0, 9.0])
point_log = [x0]
res = fsolve(func, x0=x0, args=(point_log,))
print("Solved point:", res)
print("Value at solved point:", func(res))
print("Function evaluations:", len(point_log))

xgrid = np.linspace(-15.0, 15.0, 40)
ygrid = xgrid.copy()
xmesh, ymesh = np.meshgrid(xgrid, ygrid, indexing="ij")
zmesh = np.zeros_like(xmesh).flatten()

for i, (xp, yp) in enumerate(zip(xmesh.flatten(), ymesh.flatten())):
    zmesh[i] = np.linalg.norm(func(np.array([xp, yp])))
zmesh = zmesh.reshape(xmesh.shape)

plt.figure()
plt.imshow(zmesh.T, origin="lower", extent=(-15, 15, -15, 15))
plt.colorbar()
plt.contour(xmesh, ymesh, zmesh, colors="k")
x, y = zip(*point_log)
plt.plot(x, y, marker=".", color="w")
plt.scatter([x[-1]], [y[-1]], color="r", zorder=999)

##### Signal Processing

* Signal processing combats nefarious organized noise that does not behave like statistical noise
* Zero-phase forward/backward filtering is usually the right kind for postprocessing hardware performance data
* Comparing filters

In [ ]:
# Signal Processing: Zero-Phase Filtering

import numpy as np
from scipy.signal import butter, filtfilt, dlti, dbode, dlsim

# Make some noisy data
x = np.linspace(0.0, 100.0, 10000)
y_truth = np.sin(x)
y_truth[600:] += 2.0  # Demonstrate response to step discontinuity
y_meas = y_truth + np.random.normal(0.0, 0.2, len(x))  # Noisy signal

# Filter
cutoff = 0.01
#   S-space polynomial coeffs for numerator and denominator
filt1 = butter(1, cutoff)  # First-order filter for forward-backward filter
filt2 = butter(2, cutoff)  # Second-order filter for forward filtering
filt2_norm = butter(
    2, cutoff, fs=1.0
)  # Normalized to samplerate for bode plot purposes
#   Run the filter forward (like a realtime deterministic filter)
#   To compare to a forward-backward filter, use a 2nd-order filter for the forward-only version.
y_filt = dlsim((*filt2, x[1] - x[0]), u=y_meas, t=x)[1]
#   Run filter forward then backward, cancelling phase shift
#   This is a nondeterministic filter (runs partially backward in time)
#   suitable for post-test filtering. First order in each direction
#   combines to make a second-order overall filter.
y_filtfilt = filtfilt(*filt1, y_meas)

# Frequency response
filter_sys = dlti(*filt2_norm, dt=1.0)  # normalized to samplerate
w, mag, phase = dbode(
    filter_sys,
    w=np.logspace(np.log10(cutoff / 10), np.log10(0.4), 1000) * 2.0 * np.pi,
)
f = w / (2.0 * np.pi)  # [Hz / samplerate] normalized freq

# Mag
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=((7, 4)), sharex=True)
ax1.set_xscale("log")
ax1.plot(f, mag, color="k")
ax1.set_ylabel("Gain [dB]")
ax1.axvline(cutoff, color="k", alpha=1.0)
ax1.grid(True)

# Phase
ax2.plot(f, phase, color="k")
ax2.set_ylabel("Phase [deg]")
ax2.set_xlabel("f / fs")
ax2.axhline(-90, color="k", alpha=0.4)
ax2.axvline(cutoff, color="k", alpha=1.0)
ax2.grid(True)
plt.suptitle("Butterworth Order 2 - Forward Response")

# Time-domain response
plt.figure(figsize=(12, 4))
plt.plot(x, y_meas, color="k", alpha=0.3, label="Measured")
plt.plot(x, y_truth, linewidth=2, color="k", label="Truth")
plt.plot(x, y_filt, color="b", alpha=1.0, label="Fwd Filtered")
plt.plot(x, y_filtfilt, color="r", alpha=0.6, label="Fwd-Bwd Filtered")
plt.legend()
plt.ylabel("y")
plt.xlabel("x")

# Time-domain zoom in
plt.figure(figsize=(12, 4))
plt.plot(x[200:1000], y_meas[200:1000], color="k", alpha=0.3, label="Measured")
plt.plot(x[200:1000], y_truth[200:1000], linewidth=2, color="k", label="Truth")
plt.plot(x[200:1000], y_filt[200:1000], color="b", alpha=1.0, label="Fwd Filtered")
plt.plot(
    x[200:1000], y_filtfilt[200:1000], color="r", alpha=0.6, label="Fwd-Bwd Filtered"
)
plt.legend()
plt.ylabel("y")
plt.xlabel("x")

plt.show()

In [ ]:
# Signal Processing: Comparing Filters
# NOTE: Discuss phase behavior of FIR filters & phase behavior of both filters in non-causal form
# NOTE: Discuss how number of FIR taps grows with decreasing cutoff ratio, but IIR filters stay the same
import numpy as np
from scipy.signal import butter, cheby1, savgol_coeffs, dlti, dbode, freqz


def compare_filters(cutoff_ratio, ripple, poly_order, nsav, nmean):
    #   S-space polynomial coeffs for numerator and denominator
    iir_filters = [
        (
            f"Butterworth Order {2 * (i + 1)}",
            butter(2 * (i + 1), Wn=cutoff_ratio, fs=1.0),
        )
        for i in [0, 2]
    ]
    iir_filters += [
        (
            f"Chebyshev Order {2 * (i + 1)}",
            cheby1(2 * (i + 1), ripple, Wn=cutoff_ratio, fs=1.0),
        )
        for i in [0, 2]
    ]
    #   Filter taps (just a vector of floats to convolve with the signal)
    fir_filters = []
    fir_filters += [(f"Sav-Gol Order 4 Size {nsav}", savgol_coeffs(nsav, poly_order))]
    fir_filters += [(f"Sliding Mean Size {nmean}", np.ones(nmean) / nmean)]

    f = np.logspace(-3, np.log10(0.4), 1000)
    w = f * (2.0 * np.pi)

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=((7, 4)), sharex=True)
    ax1.set_xscale("log")
    ax1.axvline(cutoff_ratio, color="k", alpha=1.0)
    ax1.grid(True)
    ax1.set_ylabel("Gain [dB]")
    ax1.set_ylim(bottom=-40)

    ax2.set_xscale("log")
    ax2.axvline(cutoff_ratio, color="k", alpha=1.0)
    ax2.grid(True)
    ax2.set_ylabel("Gain [dB]")
    ax2.set_ylim(bottom=-3)

    ax3.set_ylabel("Phase [deg]")
    ax3.set_xlabel("f / fs")
    ax3.axhline(-90, color="k", alpha=0.4)
    ax3.axvline(cutoff_ratio, color="k", alpha=1.0)
    ax3.grid(True)

    plt.suptitle(f"Filter Comparison - Cutoff Ratio = {cutoff_ratio}")

    for name, fi in iir_filters:
        filter_sys = dlti(*fi, dt=1.0)  # normalized to samplerate
        w, mag, phase = dbode(filter_sys, w=w)
        linestyle = "-" if "butter" in name.lower() else "-."
        ax1.plot(f, mag, linestyle=linestyle, label=name)
        ax2.plot(f, mag, linestyle=linestyle, label=name)
        ax3.plot(f, phase, linestyle=linestyle, label=name)

    for name, fi in fir_filters:
        _, h = freqz(fi, worN=w)
        mag = 20.0 * np.log10(np.abs(h))
        # phase = np.unwrap(np.angle(h, deg=True))  # No point examining this
        ax1.plot(f, mag, linestyle="--", label=name)
        ax2.plot(f, mag, linestyle="--", label=name)

    ax1.legend(
        ncol=2,
        loc="upper left",  # Anchor upper left corner of legend box at bbox_to_anchor
        # Anchor here in normalized plot space relative to lower left corner
        bbox_to_anchor=(1.0, 1.0),
        edgecolor="black",  # Color of legend frame
        fancybox=False,  # No rounded edges
        borderaxespad=0,  # Set padding between legend and plot frame
    )

In [ ]:
# At a cutoff ratio of 0.1, evaluation cost is fairly balanced between IIR and FIR

cutoff_ratio = 0.1  # Target cutoff freq as a fraction of samplerate
ripple = 0.1  # Allowed gain ripple before cutoff (in passband)
poly_order = 4  # Polynomial order for sav-gol
nsav = 17  # Number of Sav-Gol taps
nmean = 5  # Number of sliding-mean taps

compare_filters(cutoff_ratio, ripple, poly_order, nsav, nmean)

In [ ]:
# At a cutoff ratio of 0.01, evaluation cost is strongly in favor of IIR

cutoff_ratio = 0.01  # Target cutoff freq as a fraction of samplerate
ripple = 0.1  # Allowed gain ripple before cutoff (in passband)
poly_order = 4  # Polynomial order for sav-gol
nsav = 171  # Number of Sav-Gol taps
nmean = 45  # Number of sliding-mean taps

compare_filters(cutoff_ratio, ripple, poly_order, nsav, nmean)

In [ ]:
# Quick reference for dB vs attenuation
# NOTE: Discuss implications re: quality of filters

attenuation = np.logspace(-6, 0, 1000)
db = 20 * np.log10(attenuation)

plt.plot(db, attenuation, color="k")
plt.xlabel("dB")
plt.ylabel("Attenuation")
plt.yscale("log")
plt.grid(True)

#### FinDiff & A Quick PDE Solver

This example demonstrates setting up finite-difference differential operators and solving a 2D PDE (Grad-Shafranov axisymmetric flux solve), and compares the results to a hand-crafted solution.

Implements the operator

$$ \Delta^{*} \Psi \equiv R \frac{\partial}{\partial R} \left( \frac{1}{R} \frac{\partial \Psi}{\partial R} \right)
+ \frac{\partial^2 \Psi}{\partial Z^2}
$$

or, after a step of product rule,

$$
\Delta^{*} \Psi \equiv \frac{\partial^2 \Psi}{\partial Z^2} + \frac{\partial^2 \Psi}{\partial R^2}
                   - \frac{1}{R} \frac{\partial \Psi}{\partial R}
$$

to solve

$$
\Delta^{*} \Psi = - \mu _{0} * (2 \pi  R) * J_{tor}
$$

FinDiff develops the finite difference stencils either using stored templates (for regular grids) or, for irregular grids, by symbolically inverting a Vandermonde matrix to resolve the coefficients of a minimum-order polynomial for each footprint.

In [ ]:
from scipy import sparse

import findiff
# from findiff.grids import Grid, NonEquidistantAxis  # For irregular grids

##### Set up the grids

In [ ]:
# Set up grids
rgrid = np.linspace(0.01, 5.0, 100)  # [m]
zgrid = np.linspace(-4.0, 4.0, int(8.0 / 0.05))  # [m]
rmesh, zmesh = np.meshgrid(rgrid, zgrid, indexing="ij")  # [m]

dr = rgrid[1] - rgrid[0]  # [m]
dz = zgrid[1] - zgrid[0]  # [m]

nr, nz = len(rgrid), len(zgrid)

# For irregular grids:
# dr = np.diff(rgrid)
# dr = np.diff(rgrid, prepend=rgrid[0] - dr[0])
# dz = np.diff(zgrid)
# dz = np.diff(zgrid, prepend=zgrid[0] - dz[0])

# drmesh = np.tile(dr, nz).reshape(rmesh.shape)
# dzmesh = np.tile(dz, nr).reshape(rmesh.shape)

extent = (np.min(rgrid), np.max(rgrid), np.min(zgrid), np.max(zgrid))  # [m]

target_shape = rmesh.shape

# Build abstract differential operators
# rax = NonEquidistantAxis(0, rgrid)  # For irregular grids
# zax = NonEquidistantAxis(1, zgrid)
grid = {0: dr, 1: dz}

ddr_op = findiff.Diff(0, acc=4)  # [1/m]
ddr_op.set_grid(grid)
ddz_op = findiff.Diff(1, acc=4)  # [1/m]
ddz_op.set_grid(grid)

##### Build differential operators

In [ ]:
# Realize differential operators & build LHS
ddr = ddr_op.matrix(target_shape)  # [1/m]
ddr2 = (ddr_op**2).matrix(target_shape)  # [1/m^2]
ddz2 = (ddz_op**2).matrix(target_shape)  # [1/m^2]

# R-factors as diagonal matrices
r = sparse.diags_array(rmesh.flatten())  # [m] r as row scaling
rinv = sparse.diags_array(1 / rmesh.flatten())  # [1/m] 1/r as row scaling

# Full delta-star operator
# delta_star = r @ ddr @ rinv @ ddr + ddz2  # Form 1
delta_star = ddr2 + ddz2 - rinv @ ddr  # [1/m^2] Form 2
delta_star = delta_star.tocsr()  # CSR for setting BCs

# 'spy' can show the sparsity pattern of a large matrix without
# actualizing the whole thing
plt.spy(delta_star, color="k")
plt.title(r"$\Delta^{*}$ Operator")

##### Build right-hand-side

In [ ]:
# Build the RHS
# -2.0 * pi * mu_0 * R * jtor
from scipy.constants import mu_0

# Example current density distribution
rmid, zmid = np.mean(rgrid), np.mean(zgrid)  # [m]
radius = 0.5  # [m]
circle = lambda r, z: np.where((r - rmid) ** 2 + (z - zmid) ** 2 <= radius**2, 1.0, 0.0)
jtor = circle(rmesh, zmesh)  # [A/m^2] toroidal current density

# RHS given current density distribution
rhs = -2.0 * np.pi * mu_0 * rmesh * jtor

##### Set BCs

In [ ]:
# Set BCs in LHS
# This enforces that each point on the boundary of the solution will be held to the exact value found on the boundary in the RHS
# which allows us to send through a solution calculated using a filament formula that is the Green's Function for this system.
#
# This practice of setting the boundary values to a valid solution before the solve allows us to solve over a finite domain
# instead of needing to integrate to infinity to model the free boundary of the domain.

#    Get boundary indices
rows, cols = np.meshgrid(range(nr), range(nz), indexing="ij")
flats = np.array(range(nr * nz)).reshape(target_shape)

brows, bcols = [], []
bflats = []
for s in [(0, ...), (-1, ...), (..., 0), (..., -1)]:
    brows.extend(rows[s])
    bcols.extend(cols[s])
    bflats.extend(flats[s])

boundary_inds = (brows, bcols)

#    Set boundary rows to 1
eye = sparse.eye(*delta_star.shape, format="csr")
for i in bflats:
    delta_star[i, :] = eye[i, :]


In [ ]:
# Set BCs in RHS
from cfsem import flux_circular_filament

inds = np.where(jtor.flatten() != 0.0)  # Where there is current

# Green's function method for BC
jtor[boundary_inds] = flux_circular_filament(
    jtor.flatten()[inds],
    rmesh.flatten()[inds],
    zmesh.flatten()[inds],
    rmesh[boundary_inds],
    zmesh[boundary_inds],
)  # [Wb] we're mixing units within this array because some entries are pre-solve forcing terms, and some are post-solve BCs

##### Solve the system

In [ ]:
# Solve
solver = sparse.linalg.factorized(delta_star.tocsc())
psi = solver(rhs.flatten()).reshape(target_shape)  # [Wb]

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2)
plt.sca(axes[0])
plt.imshow(jtor.T, origin="lower", extent=extent, aspect="equal")
plt.colorbar()
plt.title("jtor [A/m^2]")

plt.sca(axes[1])
plt.imshow(psi.T, origin="lower", extent=extent, aspect="equal")
plt.gca().contour(psi.T, levels=6, colors="k", linewidths=1, extent=extent)
plt.colorbar()
plt.title("psi [Wb]")

##### Compare to reference implementation

In [ ]:
from cfsem import gs_operator_order4

# Get operator with BCs already set
val, row, col = gs_operator_order4(rgrid, zgrid)
delta_star_cfsem = sparse.coo_matrix((val, (row, col))).tocsc()

# Solve
solver_cfsem = sparse.linalg.factorized(delta_star_cfsem)
psi_cfsem = solver_cfsem(rhs.flatten()).reshape(target_shape)  # [Wb]

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
plt.sca(axes[0])
plt.imshow(jtor.T, origin="lower", extent=extent, aspect="equal")
plt.colorbar()
plt.title("jtor [A/m^2]")


plt.sca(axes[1])
plt.imshow(psi_cfsem.T, origin="lower", extent=extent, aspect="equal")
plt.gca().contour(psi_cfsem.T, levels=6, colors="k", linewidths=1, extent=extent)
plt.colorbar()
plt.title("psi (cfsem) [Wb]")

err = psi.T - psi_cfsem.T
plt.sca(axes[2])
plt.imshow(err, origin="lower", extent=extent, aspect="equal")
plt.colorbar()
plt.title("psi err [Wb]")

----
#### Plotting

* Matplotlib is the most popular & mirrors matlab's plotting API
  * Good for publications, posters, etc
  * Only interactive while program is running
* Seaborn provides better styling for matlab
* Plotly provides HTML plots
  * Good for persistent documentation that retain interactivity after program exits
  * Also able to generate static plots when needed

##### Styling w/ Seaborn

In [ ]:
import seaborn as sns

sns.set_context("paper", font_scale=1.4)
# NOTE: Try other styles (white, whitegrid, ticks, dark, darkgrid)
sns.set_style("ticks")
sns.set_palette("cubehelix")

fig = plt.figure(figsize=(6, 4))
plt.title("Basic paper-style plots")
plt.plot(x, y_filtfilt, linewidth=1.5)

plt.figure(figsize=(6, 4))
plt.title("Too smooth? Turn off antialiasing")
plt.plot(x, y_filtfilt, linewidth=1.0, antialiased=False)

##### Vector Graphics

In [ ]:
# Display SVG in a notebook for crisper plots
# %config InlineBackend.figure_format = 'svg'

# Save figures as SVG or EPS (vector graphics formats)
# to keep that crispness forever
fig = plt.figure(figsize=(6, 4))

plt.plot(x, y_filtfilt, linewidth=1.5)

svg_path = static_dir / "paper_style.svg"
plt.savefig(svg_path)

plt.title("Use SVG for crisp lines at any scale\n(uncomment the %config line)")
plt.show()

##### Example: Fill-Between

In [ ]:
plt.figure()
y0 = y_filt.flatten()[:1000]
y1 = y_filtfilt.flatten()[:1000]
plt.plot(x[:1000], y0, color="k", linewidth=1.0)
plt.plot(x[:1000], y1, color="k", linewidth=1.0)
plt.fill_between(x[:1000], y0, y1, color="k", alpha=0.2)

##### Plotly

Plotly generates interactive HTML plots, which are typically not the kind used for a publication, but preserves pan, zoom, and mouse-hover inspection.

In [ ]:
import plotly.express as px  # Shortcut API
from plotly import graph_objects as go  # Detailed API

# "express" shorthand API
px.line(x=x[:1000], y=y0)


In [ ]:
# Detailed API can be massaged to produce matplotlib-like output
fig = go.Figure(data=[go.Scatter(x=x, y=y_filtfilt, line={"color": "black"})])
fig.update_layout(plot_bgcolor="white")
fig.update_xaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    # gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    # gridcolor='lightgrey'
)
fig.show()

In [ ]:
# Plotly can also export SVG output
# This requires the additional "kaleido" dep that is not installed by default
fig.write_image(static_dir / "paper_style_plotly.svg")

In [ ]:
# 3D scatter plots - Plotly is much better for this than matplotlib
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x[:1000], y=y0, z=y1, line={"color": "black"}, marker={"size": 2}
        )
    ]
)
fig.show()

##### Accessibility

Using distinct combinations of markers and colors that have distinct intensity can be helpful for the ~4% of the population that has some kind of colorblindness. It's also helpful for everyone else.

To do this,

* Use an intensity-mapped color palette like "cubehelix" or "colorblind"
* Use a cycle of markers that are distinct independent of color
* Make the color and marker cycles' length not an integer multiple of each other
  * Or make a cycle over the cartesian product of both like `cycle(product(colors, markers))`
  * Offset cycle makes a better pattern
* Choose a number of markers you'd like to see per trace (usually around 5-10) and only plot those instead of every marker
* Offset sequential markers from each other so that they avoid overlaps when possible

There are also some more helpful habits that can improve the usability of plots in general
* Use a font size that is readable from the same distance where the plot data is readable - shouldn't have to squint
* Place the legend outside the plot so that people can see what is going on in both the legend and the plot
* Use markers that can be referred to easily by name
  * This way, discussion can happen without struggling through the "no the other one that's kind of a rotated square - not that one, that's a diamond - no, the filled one, not the outline - ..."
  * Letters of the alphabet work well for this, as long as you only take one from each confusing group like `I / J / L`, `Q / O / D`, and `M / N / W / V` that can be difficult to distinguish in a busy plot

In [ ]:
# Matplotlib accessibility formatting

from itertools import cycle

colors = cycle(sns.cubehelix_palette(n_colors=10))
# colors = cycle(sns.color_palette("colorblind", n_colors=7))  # Another good option
markers = cycle(["$A$", "$B$", "$C$", "$D$", "$E$", "$F$", "$G$", "$H$", "$K$"])

nmarkers = 6  # Target number of markers in-frame for each trace
n = len(x)  # Total number of data points
nskip = int(n // nmarkers)  # Number of indices to skip between markers
noffs = max(int(nskip / 10), 1)  # Amount to offset the start of each series of markers

plt.figure()
for i in range(20):
    # Get the next marker/color combo
    c = next(colors)
    m = next(markers)

    # Draw the line
    yi = np.sin(i * x / 200)
    plt.plot(x, yi, color=c, linewidth=2)

    # Mark every nth data point, starting at an offset to avoid overlapping markers
    marker_start = (i * noffs) % nskip
    plt.scatter(
        x[marker_start::nskip], yi[marker_start::nskip], color=c, marker=m, label=f"{i}"
    )

plt.legend(
    ncol=3,
    loc="upper left",  # Anchor upper left corner of legend box at bbox_to_anchor
    bbox_to_anchor=(
        1.0,
        1.0,
    ),  # Anchor here in normalized plot space relative to lower left corner
    edgecolor="black",  # Color of legend frame
    fancybox=False,  # No rounded edges
    borderaxespad=0,  # Set padding between legend and plot frame
)

In [ ]:
# Plotly accessibility formatting

from itertools import cycle

colors = cycle(sns.cubehelix_palette(n_colors=10).as_hex())
# colors = cycle(sns.color_palette("colorblind", n_colors=7))  # Another good option
markers = cycle(
    [
        "circle",
        "square",
        "diamond",
        "cross",
        "triangle-up",
        "triangle-down",
        "triangle-left",
        "triangle-right",
    ]
)

nmarkers = 6  # Target number of markers in-frame for each trace
n = len(x)  # Total number of data points
nskip = int(n // nmarkers)  # Number of indices to skip between markers
noffs = max(int(nskip / 10), 1)  # Amount to offset the start of each series of markers

data = []
for i in range(20):
    # Get the next marker/color combo
    c = next(colors)
    m = next(markers)

    # Draw the line
    yi = np.sin(i * x / 200)
    data.append(
        go.Scatter(x=x, y=yi, line={"color": c}, mode="lines", showlegend=False)
    )

    # Mark every nth data point, starting at an offset to avoid overlapping markers
    marker_start = (i * noffs) % nskip
    data.append(
        go.Scatter(
            x=x[marker_start::nskip],
            y=yi[marker_start::nskip],
            marker={"color": c, "symbol": m, "size": 8},
            mode="markers",
            name=f"{i}",
        )
    )

fig = go.Figure(data=data)
fig.update_layout(
    plot_bgcolor="white",
    # Horizontal legend above plot will wrap into multiple lines,
    # but vertical legend to the side will not wrap to multiple columns
    legend=dict(
        entrywidth=0.1,
        entrywidthmode="fraction",
        orientation="h",
        y=1.2,
        xanchor="center",
        x=0.5,
    ),
)
fig.update_xaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    # gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    # gridcolor='lightgrey'
)
fig.update_xaxes(range=[np.min(x), np.max(x)])
fig.show()

##### Example - Moody Chart

A common reference chart in fluid mechanics is the Moody chart, which shows pipe flow friction factor (used to calculate how much pressure is required to drive a given flowrate) as a function of Reynold's Number ($Re = \frac{\rho * v * D_h}{\mu}$) and relative roughness ($\epsilon = \frac{\text{surface roughness}}{D_h}$).

There are a variety of methods for calculating the friction factor, and they produce different results for a given $Re$ and $\epsilon$. There are also a wide range of methods for calculating what constitutes "surface roughness" since surface profiles come in many forms that affect flow differently.

In [ ]:
re = np.logspace(
    2.8, 8, 300
)  # [dimensionless] Reynold's Number w.r.t. hydraulic diameter
roughness = [
    1e-5,
    1e-4,
    1e-3,
    1e-2,
    0.05,
    0.1,
]  # [dimensionless] relative roughness w.r.t. hydraulic diameter

TURBULENT_CUTOFF = 4000.0  # [dimensionless] Rule-of-thumb Re where a pipe flow should be fully turbulent
TRANSITION_CUTOFF = (
    2300.0  # [dimensionless] Rule-of-thumb Re where laminar-turbulent transition starts
)


# First method is implicit -> expensive and only covers turbulent region
def f_with_colebrook(re: NDArray, roughness: float) -> NDArray:
    """
    A friction factor calculation using the implicit Colebrook eqn for the turbulent regime
    and a linear weighted average blending to the analytic laminar calc through the transition region.
    """
    out = np.array([f_colebrook(x, roughness) for x in re])
    out_lam = 64.0 / re  # Laminar friction factor is a very simple formula

    # Find regions
    # turb = np.where(re > TURBULENT_CUTOFF)
    lam = np.where(re < TRANSITION_CUTOFF)
    transitional = np.where((re >= TRANSITION_CUTOFF) & (re <= TURBULENT_CUTOFF))

    # Make blended transition region
    weight = (re[transitional] - TRANSITION_CUTOFF) / (
        TURBULENT_CUTOFF - TRANSITION_CUTOFF
    )
    out[transitional] = (
        weight * out[transitional] + (1.0 - weight) * out_lam[transitional]
    )
    # Make laminar region
    out[lam] = out_lam[lam]

    return out


# Second method is explicit and covers full range from laminar to fully turbulent
def f_bellos(re: NDArray, roughness: float):
    """
    An explicit correlation of Darcy friction factor for pipe flow
    with Reynolds number and surface roughness.

    Arguments:
        re: Reynolds number, with hydraulic diameter as the length scale [units: dimensionless].
        roughness: Pipe inside surface roughness scale height divided by pipe diameter [units: dimensionless].

    Returns:
        Darcy friction factor [units: dimensionless].

    References:
        [1] Vasilis Bellos, Ioannis Nalbantis, and George Tsakiris, "Friction Modeling of Flood Flow Simulations,"
            Journal of Hydraulic Engineering, Dec 2018.
            https://doi.org/10.1061%2F%28asce%29hy.1943-7900.0001540
        [2] Vasilis Bellos, Ioannis Nalbantis, and George Tsakiris, "Erratum for 'Friction Modeling of Flood Flow Simulations',"
            Journal of Hydraulic Engineering, Oct 2020.
    """
    a = 1 / (1 + (re / 2712.0) ** 8.4)
    b = 1 / (1 + (re * roughness / 150.0) ** 1.8)
    f = (
        (64 / re) ** a
        * (0.75 * np.log(re / 5.37)) ** (2 * (a - 1) * b)
        * (0.88 * np.log(3.41 / roughness)) ** (2 * (a - 1) * (1 - b))
    )
    return f


In [ ]:
# Calculate and plot Moody charts of both calc methods
for name, func in [
    ("Implicit Colebrook", f_with_colebrook),
    ("Explicit Bellos", f_bellos),
]:
    data = []
    for e in roughness:
        # Draw the line
        yi = func(re, e)
        data.append(
            go.Scatter(
                x=re, y=yi, line={"color": "black"}, mode="lines", showlegend=False
            )
        )

        # Mark the end of each trace
        data.append(
            go.Scatter(
                x=[re[-1]],
                y=[yi[-1]],
                mode="text",
                text=f"D/ε={1 / e:.0f}",
                textposition="middle right",
                showlegend=False,
            )
        )

    fig = go.Figure(data=data)
    fig.update_layout(
        plot_bgcolor="white",
        title=name,
    )
    fig.update_xaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        # gridcolor='lightgrey'
        type="log",
        title="Re [dimensionless]",
    )
    fig.update_yaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        # gridcolor='lightgrey'
        title="friction factor [dimensionless]",
    )
    fig.update_xaxes(range=[np.log10(np.min(re)), np.log10(np.max(re) + 3e8)])
    fig.show()

----
#### Autodiff & Optimization Frameworks

* [Casadi](https://web.casadi.org/) - excellent for **sparse systems** (typical for controls)
  * **First-class support for solvers and constraints**
  * Uses either interior point or SQP nonlinear solver under the hood w/ excellent method for Jacobian traversal
  * Significantly enhanced by using convenience methods from [aerosandbox](https://github.com/peterdsharpe/AeroSandbox)
  * Can do codegen in C for simple systems (intended to support embedded control systems)
  * Medium software build quality - could use better type hinting and such
  * Fantastic support for **arbitrary nonlinear constraints** on any part of the expression (inputs, intermediate states, or outputs)
    * Incredibly powerful tool for constrained design optimization or simulation!
* [Jax](https://docs.jax.dev/en/latest/quickstart.html) - excellent for **dense systems** (typical for matrix-free PDE solvers and ML)
  * Essentially **no native support for constraints or optimization - only differentiation**
    * Can be used as a backend for functions in Casadi! Casadi accepts plugins from anywhere - this gives a quick way to bring Jax capabilities into a proper solver framework
  * Provides a JIT (just-in-time compiler) in addition to differentiation capability
  * Not clever about Jacobian traversal - always either forward or backward (this is good enough for many things)
  * Excellent software build quality (it's a google product)
* [CVXPY](https://www.cvxpy.org/) - for large or discrete, sparse or dense, **strictly convex systems** (linear, and geometric programs) or systems having a strictly convex linear relaxation (mixed-integer linear)
  * **First-class support for solvers and constraints**
  * Can solve **enormous systems** with **40,000+ variables**
  * Primarily for **disciplined convex systems** - not general nonlinear capability like Casadi
  * Mixed-integer programs include operations planning algorithms (for example, scheduling shifts or machine time, packing, shipping/delivery, etc)
* [tensorflow](https://www.tensorflow.org/) and [scikit-learn](https://scikit-learn.org/stable/) - machine learning only
  * Honorable mention because they do some autodiff activities under the hood, but aren't as useful for more general problem-solving
  

A quick primer on what these do -

##### The goal

**Solve systems of nonlinear equations**, usually phrased as an **optimization** or **root-finding** problem.

This class of problem includes 

* Engineering simulations (ODE & DAE integrators, algebraic equilibrium problems, etc)
* Design optimization
* Optimal control
* Machine learning & curvefitting

and anything else that you can phrase as a system of equations.

##### Under the hood

The first and most essential function of an autodiff system is to **calculate the gradient of a function or the Jacobian matrix of a system**, which is the matrix of first derivatives - the  sensitivities of all of the outputs to all of the inputs.

Given some vector function $\mathbf{y} = f(\mathbf{x})$, the Jacobian is

$J(\mathbf{x})
=\begin{bmatrix}
\displaystyle\frac{\partial y_{1}}{\partial x_{1}} & \displaystyle\frac{\partial y_{1}}{\partial x_{2}} & \cdots & \displaystyle\frac{\partial y_{1}}{\partial x_{n}}\\[1em]
\displaystyle\frac{\partial y_{2}}{\partial x_{1}} & \displaystyle\frac{\partial y_{2}}{\partial x_{2}} & \cdots & \displaystyle\frac{\partial y_{2}}{\partial x_{n}}\\
\vdots & \vdots & \ddots & \vdots\\
\displaystyle\frac{\partial y_{m}}{\partial x_{1}} & \displaystyle\frac{\partial y_{m}}{\partial x_{2}} & \cdots & \displaystyle\frac{\partial y_{m}}{\partial x_{n}}
\end{bmatrix}$

The gradient of each individual output forms one column of the Jacobian.

The Jacobian can be built either "forward" (on column at a time) by calculating forward sensitivites to each input, or "backward" (one row at a time) by accumulating the sensitivity of each output back to the inputs using repeated application of the chain rule.
* Forward differentiation: Low-memory, good for cases with **few inputs** and **many outputs** - columns of J are larger than rows
* Reverse differentiation: High-memory, good for cases with **many inputs** and **few outputs** - rows of J are larger than columns

For a large function, the full Jacobian can be calculated **partly in forward mode** and **partly in reverse mode**. Choosing the best possible mix of forward- and reverse-mode accumulation is called the **Jacobian Traversal Problem** and is known to be **NP-hard** - it is unreasonable to look for an optimal solution, but there are some good heuristics.

Autodiff frameworks allow composing the calculation of gradients and jacobians, which allows calculating the **Hessian matrix** - the matrix of second derivatives (the Jacobian of the gradient). Because this is a very expensive 3D result for functions with multiple outputs, it is usually applied to a **cost function** that rolls many contributions up into a **single output** so that the resulting Hessian matrix is 2D.

$H(\mathbf{x}) = J(\nabla f(\mathbf{x})) =
\begin{bmatrix}
\displaystyle
\frac{\partial^2 f}{\partial x_1^2}
& 
\displaystyle
\frac{\partial^2 f}{\partial x_1\,\partial x_2}
& 
\cdots 
& 
\displaystyle
\frac{\partial^2 f}{\partial x_1\,\partial x_n}
\\[1em]
\displaystyle
\frac{\partial^2 f}{\partial x_2\,\partial x_1}
& 
\displaystyle
\frac{\partial^2 f}{\partial x_2^2}
& 
\cdots
& 
\displaystyle
\frac{\partial^2 f}{\partial x_2\,\partial x_n}
\\[0.75em]
\vdots & \vdots & \ddots & \vdots
\\[0.75em]
\displaystyle
\frac{\partial^2 f}{\partial x_n\,\partial x_1}
&
\displaystyle
\frac{\partial^2 f}{\partial x_n\,\partial x_2}
&
\cdots
&
\displaystyle
\frac{\partial^2 f}{\partial x_n^2}
\end{bmatrix}.
$

##### Further Reading

* [1] M. Saroufim, “Automatic Differentiation Step by Step,” Medium. Accessed: Jun. 06, 2021. [Online]. Available: https://marksaroufim.medium.com/automatic-differentiation-step-by-step-24240f97a6e6

* [2] J. Hückelheim, H. Menon, W. Moses, B. Christianson, P. Hovland, and L. Hascoët, “Understanding Automatic Differentiation Pitfalls,” May 12, 2023, arXiv: arXiv:2305.07546. doi: 10.48550/arXiv.2305.07546.

* [3] U. Naumann, “Optimal Jacobian accumulation is NP-complete,” Math. Program., vol. 112, no. 2, pp. 427–441, Nov. 2007, doi: 10.1007/s10107-006-0042-z.

* More about expression graphs: https://docs.sympy.org/latest/tutorials/intro-tutorial/manipulation.html

**... along with the source code of any of the open-source frameworks!**

##### Example - Optimal Charging of an Inductor w/ Series Varistor & Parallel Resistor

...subject to limits on terminal voltage, current, and dissipated power in a parallel resistive path.

```text
                   => i1 =>          ^
        vi                v1        /        vo=0
         +--------[ L ]--------[ R1 ]--------+
         |                      /            |
---------+                                   +--------
         |                                   |
         +-------------[ R2 ]----------------+
                   => i2 =>
```

In [ ]:
import numpy as np
from aerosandbox.optimization.opti import Opti  # This is casadi
import aerosandbox.numpy as anp
import casadi as ca

# This solver class will manage the expression graph,
# use it to generate the function, Jacobian, and Hessian of the objective and constraints,
# and hand them to an optimizer backend.
solver: Opti = Opti()
solver.solver("ipopt")  # Use a general-purpose interior-point solver

# Operating limits
max_voltage = 1.0  # [V]
max_current = 1e3  # [A]
max_stored_energy = 12e3  # [J]
max_dissipated_power = 200.0  # [W]

# Physical parameters
inductance = 0.1  # [H]
#   [ohm] simple variable resistor that increases resistance with increasing current
ref_current = 1.0  # [A]
r1_func = lambda current: 3e-9 * (current / ref_current) ** 2 + 1e-6
#   A normal resistor
r2 = 0.01  # [ohm] parallel

# Fixed time-grid
n = 300
time: ca.MX = solver.parameter(np.linspace(0.0, 100.0, n))

# Decision variables
terminal_voltage: ca.MX = solver.variable(np.zeros(n))  # [V] vi - vo
i1: ca.MX = solver.variable(np.linspace(0.0, 100.0, n))  # [A] inductor current

# Physics
#   Lower path
i2: ca.MX = terminal_voltage / r2  # [A] parallel current
#   Upper path V = L*dI/dt + I*R(I)
r1: ca.MX = r1_func(i1)  # Variable resistor
inductor_voltage: ca.MX = inductance * solver.derivative_of(
    i1, with_respect_to=time, derivative_init_guess=1.0, method="backward euler"
)  # V = L*dI/dt
solver.subject_to((i1 * r1 + inductor_voltage) == terminal_voltage)
#   Total
terminal_current: ca.MX = i1 + i2
stored_energy: ca.MX = 0.5 * inductance * i1**2  # [J]
dissipated_power: ca.MX = i1**2 * r1 + i2**2 * r2  # [W]
#   Initial conditions
solver.subject_to(i1[0] == 0.0)
solver.subject_to(i2[0] == 0.0)
solver.subject_to(terminal_current[0] == 0.0)
solver.subject_to(terminal_current >= 0.0)
solver.subject_to(terminal_voltage >= 0.0)

# Operating constraints
solver.subject_to((terminal_current / max_current) ** 2 < 1.0)  # Smooth absolute value
solver.subject_to((terminal_voltage / max_voltage) ** 2 < 1.0)
solver.subject_to(stored_energy < max_stored_energy)  # These are always positive
solver.subject_to(dissipated_power < max_dissipated_power)

# Objective
solver.maximize(anp.sum(i1**2))  # Charge inductor as quickly as possible

# Solve the system
sol = solver.solve()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("paper", font_scale=1.0)
sns.set_style(
    "ticks"
)  # NOTE: Try other styles (white, whitegrid, ticks, dark, darkgrid)
sns.set_palette("colorblind")


def legend():
    plt.legend(
        loc="upper left",  # Anchor upper left corner of legend box at bbox_to_anchor
        bbox_to_anchor=(
            1.0,
            1.0,
        ),  # Anchor here in normalized plot space relative to lower left corner
        edgecolor="black",  # Color of legend frame
        fancybox=False,  # No rounded edges
        borderaxespad=0,  # Set padding between legend and plot frame
    )


plt.figure(figsize=(8, 2))
plt.title("Current [A]")
plt.xlabel("time [s]")
plt.plot(sol(time), sol(terminal_current), label="Terminal")
plt.plot(sol(time), sol(i1), label="Inductor")
plt.plot(sol(time), sol(i2), label="Parallel Path")
legend()

plt.figure(figsize=(8, 2))
plt.title("Normalized Limits")
plt.plot(sol(time), sol(terminal_voltage) / max_voltage, label="Voltage")
plt.plot(sol(time), sol(terminal_current) / max_current, label="Current")
plt.plot(
    sol(time), sol(stored_energy) / max_stored_energy, label="Stored Energy"
)
plt.plot(
    sol(time),
    sol(dissipated_power) / max_dissipated_power,
    label="Dissipated Power",
)
plt.xlabel("time [s]")
legend()

plt.figure(figsize=(8, 2))
plt.title("Variable Resistor [ohm]")
plt.plot(sol(time), sol(r1))
plt.xlabel("time [s]")

plt.figure(figsize=(8, 2))
plt.title("Stored Energy [J]")
plt.plot(sol(terminal_current), sol(stored_energy))
plt.xlabel("terminal current [A]")

----

#### Symbolics

`sympy` provides a powerful system for automating and cross-checking algebra and calculus.

##### Example: Symbolic Calculation of Lagrange Polynomial Coeffs

In [ ]:
import sympy
import numpy as np

# NOTE: Discuss relation to least-squares and maximum-likelihood fitting
# NOTE: This is a special case of kernelized regression
# specialized for (1) polynomial of order N
# and for         (2) an exact solution (instead of least-squares)


def lagrange_direct(grid: list[float] | list[sympy.Symbol], loc: float | sympy.Symbol):
    """
    Develop minimum-order Lagrange polynomial interpolation stencil
    for this grid to evaluate the interpolant at `loc` given some
    values at each grid point.

    Symbolic evaluation with integer exponents allows full-precision evaluation of the coefficients,
    compared to the cumulative-product method which develops several steps of float roundoff.

    Grid locations and the interpolation location `loc` can be either numeric or symbolic.
    The values at the grid points are always symbolic and represented by `y_{i}` in the output.
    """
    n = len(grid)  # Stencil size = polynomial order + 1
    assert n < 10, "This will not work well for extremely high-order polynomials"

    # Symbolic values at grid points
    grid_vals = sympy.Matrix([sympy.symbols(f"y_{i}") for i in range(n)])

    # Build square Vandermonde matrix
    # [1 x0 x0^2 x0^3 ... x0^n-1]
    # [     ...                 ]
    # [1 xn xn^2 xn^3 ... xn^n-1]
    # Small matrix, doesn't matter if we loop
    a = np.zeros((n, n), dtype=object)
    for i in range(n):
        for j in range(n):
            a[i, j] = grid[i] ** j

    amat = sympy.Matrix(a)

    # Solve symbolic minimum-order polynomial coeffs
    # We can take the direct inverse here because it's a square matrix,
    # but if we were making a best-fit from a rectangular matrix, we'd use a pseudoinverse instead,
    # which also works just fine in sympy
    poly_coeffs = amat.inv() @ grid_vals

    # Evaluate
    # NOTE: If we were making a finite difference stencil, we could take
    # the derivative w.r.t. a symbolic `loc` here!
    # NOTE: We could also integrate w.r.t. a symbolic `loc` between two points
    v = sum(
        [poly_coeffs[i] * loc**i for i in range(n)]
    )  # The interpolated value at `loc`

    # Extract coefficients of resulting linear system
    linear_coeffs = np.array([sympy.diff(v, gv) for gv in grid_vals])

    return poly_coeffs, linear_coeffs, grid_vals, v


In [ ]:
# NOTE: Show how result varies with changing loc, especially for integer values
# NOTE: Explain why it's interesting that this turns polynomial interpolation into a dot product
# NOTE: Explain why integration over a domain or differentiation at a point also produce a result of the same form

# Symbolic inputs
# grid = [sympy.symbols(f"x_{i}") for i in range(4)]
# loc = sympy.symbols("loc")

# Numeric inputs
grid = np.array([0.0, 0.8, 1.2, 3.0])
loc = 0.5

poly_coeffs, linear_coeffs, grid_vals, v = lagrange_direct(grid, loc)
print("Symbolic interpolated value:", v)
print("Linear coeffs for polynomial interp:", linear_coeffs)

In [ ]:
# Visual explanation -
vals = np.random.uniform(-1.0, 1.0, grid.size)  # Values at grid points
xobs = np.linspace(grid[0], grid[-1], 100)  # Observation points to examine
coeffs = np.array(
    [lagrange_direct(grid, x)[1] for x in xobs]
)  # Interp coefficients for each point to examine
yobs = coeffs @ vals  # Polynomial interpolation is a linear system now!

plt.plot(xobs, yobs, color="k", label="Fine observation points")
plt.scatter(grid, vals, color="r", label="Control points")
plt.scatter(
    [loc],
    [np.dot(linear_coeffs, vals)],
    color="b",
    marker="s",
    label="Observation at `loc`",
)
plt.legend()

----
#### Fluid Properties

Like electrons, but with a 2D space of flavors, and brutally nonlinear

In [ ]:
import CoolProp.CoolProp as cp
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt

fluid = "Helium"
# fluid = "REFPROP::HELIUM"  # If we had refprop available
# NOTE: CoolProp is missing supercritical Cp bump below 5.2K
temp_range = [
    6.0,
    20.0,
]  # [K], default range doesn't go as low temp as one would like
pressure_range = [1e5, 35e5]  # [Pa]

# Specific enthalpies in J/kg-K
h_low: float = cp.PropsSI("H", "T", temp_range[0], "P", pressure_range[0], fluid)
h_high: float = cp.PropsSI("H", "T", temp_range[1], "P", pressure_range[1], fluid)
h_range = [h_low, h_high]

# Grids & meshgrids
n = 100
pgrid = np.linspace(*pressure_range, n)
hgrid = np.linspace(h_low, h_high, n)
pmesh, hmesh = np.meshgrid(pgrid, hgrid, indexing="ij")

# Input names from http://www.coolprop.org/coolprop/HighLevelAPI.html#parameter-table
prop_map = {
    "s": "S",  # [J/kg-K] specific entropy
    "rho": "D",  # [kg/m^3] mass density
    "mu": "VISCOSITY",  # [Pa-s] dynamic viscosity
    "cp": "CPMASS",  # [J/kg-K] specific heat at constant pressure
    "cv": "CVMASS",  # [J/kg-K] specific heat at constant volume
    "Rm": "gas_constant",  # [J/mol-K] molar gas constant
    "M": "molar_mass",  # [kg/mol]
    "T": "T",  # [K]
    "prandtl": "Prandtl",  # [dimensionless]
    "k": "CONDUCTIVITY",  # [W/m-K]
}


def phprop(p: float, h: float, prop: str) -> float:
    """Convenience function for scalar P-h lookup of a property on the selected fluid"""
    return cp.PropsSI(prop_map[prop], "P", p, "H", h, fluid)


def phtable(prop: str) -> NDArray:
    """Make a 2D P-h table of a property"""
    out = np.zeros_like(pmesh.flatten())
    for i, (p, h) in enumerate(zip(pmesh.flatten(), hmesh.flatten())):
        out[i] = phprop(p, h, prop)
    return out.reshape(pmesh.shape)


# Realize tabulated properties on a grid
prop_tables = {prop: phtable(prop) for prop in prop_map.keys()}

# Add commonly-used derived values
prop_tables["gamma"] = (
    prop_tables["cp"] / prop_tables["cv"]
)  # [dimensionless] Ratio of specific heats
prop_tables["R"] = prop_tables["Rm"] / prop_tables["M"]  # [J/kg-K] Mass gas constant
prop_tables["nu"] = (
    prop_tables["mu"] / prop_tables["rho"]
)  # [m^2/s] Kinematic viscosity
prop_tables["alpha"] = np.sqrt(
    prop_tables["gamma"] * prop_tables["R"] * prop_tables["T"]
)  # [m/s] speed of sound

# Error on NaN and +/-Infinity values
for k, v in prop_tables.items():
    if not np.all(v == v):
        raise ValueError(f"Nonfinite value detected in `{k}`")

In [ ]:
plt.figure()
plt.title(f"Prandtl Number\nTemp. Range {temp_range} [K]")
cs = plt.contour(pmesh, hmesh, prop_tables["prandtl"], colors="k")
plt.clabel(cs, fmt=lambda x: f"Pr={x:.3f}")

cs = plt.contour(pmesh, hmesh, prop_tables["T"], linestyles="--", colors="k")
plt.clabel(cs, fmt=lambda x: f"T={x:.1f}K")
plt.xlabel("Pressure [Pa]")
plt.ylabel("Enthalpy [J/kg-K]")

plt.figure()
plt.title(f"Speed of Sound\nTemp. Range {temp_range} [K]")
cs = plt.contour(pmesh, hmesh, prop_tables["alpha"], colors="k")
plt.clabel(cs, fmt=lambda x: r"$\alpha$" + f"={x:.0f}m/s")
plt.xlabel("Pressure [Pa]")
plt.ylabel("Enthalpy [J/kg-K]")

plt.figure()
plt.title(f"Entropy\nTemp. Range {temp_range} [K]")
cs = plt.contour(pmesh, hmesh, prop_tables["s"], colors="k")
plt.clabel(cs, fmt=lambda x: f"s={x:.1e} J/kg-K")
plt.xlabel("Pressure [Pa]")
plt.ylabel("Enthalpy [J/kg-K]")

plt.figure()
plt.title(f"Thermal Conductivity\nTemp. Range {temp_range} [K]")
cs = plt.contour(pmesh, hmesh, prop_tables["k"], colors="k")
plt.clabel(cs, fmt=lambda x: f"k={x:.3f} W/m-K")
plt.xlabel("Pressure [Pa]")
plt.ylabel("Enthalpy [J/kg-K]")

plt.figure()
plt.title(f"Density\nTemp. Range {temp_range} [K]")
cs = plt.contour(pmesh, hmesh, prop_tables["rho"], colors="k")
plt.clabel(cs, fmt=lambda x: f"rho={x:.1f} kg/m^3")
plt.xlabel("Pressure [Pa]")
plt.ylabel("Enthalpy [J/kg-K]")

plt.figure()
plt.title(f"Cp\nTemp. Range {temp_range} [K]")
cs = plt.contour(pmesh, hmesh, prop_tables["cp"], colors="k")
plt.clabel(cs, fmt=lambda x: f"Cp={x:.1f} J/kg-K")
plt.xlabel("Pressure [Pa]")
plt.ylabel("Enthalpy [J/kg-K]")

plt.figure()
plt.title(f"Gamma\nTemp. Range {temp_range} [K]")
cs = plt.contour(pmesh, hmesh, prop_tables["gamma"], colors="k")
plt.clabel(cs, fmt=lambda x: f"gamma={x:.1f} []")
plt.xlabel("Pressure [Pa]")
plt.ylabel("Enthalpy [J/kg-K]")

plt.figure()
plt.title(f"Viscosity\nTemp. Range {temp_range} [K]")
cs = plt.contour(pmesh, hmesh, prop_tables["mu"], colors="k")
plt.clabel(cs, fmt=lambda x: f"mu={x:.1e} [Pa-s]")
plt.xlabel("Pressure [Pa]")
plt.ylabel("Enthalpy [J/kg-K]")

In [ ]:
# Making differentiable fluid property tables for Casadi
import casadi as ca
from scipy.interpolate import RectBivariateSpline


def cas_interp(name: str, grids: list[NDArray], vals: NDArray) -> ca.MX:
    """Build a differentiable B-spline interpolator"""
    return ca.interpolant(name, "bspline", grids, vals.ravel(order="F"))


# Make interpolators over every property
#  Casadi
prop_cas_interps: dict[str, ca.MX] = {
    prop: cas_interp(prop, [pgrid, hgrid], table) for prop, table in prop_tables.items()
}
#  Scipy
prop_sp_interps: dict[str, RectBivariateSpline] = {
    prop: RectBivariateSpline(pgrid, hgrid, table)
    for prop, table in prop_tables.items()
}

# Check interpolators for consistency
mid = (np.mean(pgrid), np.mean(hgrid))
print("Differentiable interpolator:", prop_cas_interps["T"]([*mid]))
print("Numeric interpolator:", prop_sp_interps["T"](*mid, grid=False))

#### Example: Subsonic Cold Helium Pipe Flow

...ignoring entrance effects & assuming subsonic & no heat transfer

with constant area

fully-developed everywhere

at steady-state

In [ ]:
# NOTE: Discuss rephrasing for design optimization and/or to include heat transfer
# NOTE: Show discretization sensitivity
# NOTE: Show sensitivity to inlet pressure & discuss tradeoff of incompressible static==stagnation assumption
# NOTE: Discuss uncertainty in fluid properties
import numpy as np
from aerosandbox.optimization.opti import Opti
import casadi as ca

solver: Opti = Opti()
solver.solver("ipopt")

# Problem setup
# Using a small pressure delta to avoid creating a Fanno flow https://en.wikipedia.org/wiki/Fanno_flow
length = 100.0  # [m] length of duct
dh = 4e-3  # [m] hydraulic diameter
print(f"Hydraulic diameter: {dh:.4f}")
area = np.pi * (dh / 2) ** 2  # [m^2] duct flow area
pi = 28e5  # [Pa] inlet stagnation pressure
po = 14e5  # [Pa] outlet stagnation pressure
ti = 15.0  # [K] inlet stagnation temperature
# [J/kg] One stagnation enthalpy - isenthalpic flow (lossy without heat transfer)
hi = cp.PropsSI("H", "T", ti, "P", pi, fluid)
roughness = 1e-6 / dh  # [dimensionless] relative roughness

# Grid
n = 100  # Number of segments
xgrid = np.linspace(0.0, length, n)
x: ca.MX = solver.parameter(xgrid)  # [m]

# Decision variable
mdot: ca.MX = solver.variable(0.01)  # [kg/s]
p: ca.MX = solver.variable(np.linspace(pi, po, n))  # [Pa] stagnation pressure

# Initialize enthalpy and static pressure
ps: ca.MX = solver.variable(np.linspace(pi, po, n))  # [Pa] static pressure
hs: ca.MX = solver.variable(hi * np.ones(n))  # [J/kg] static enthalpy

# Thermo
#   Section average relations
h: ca.MX = solver.parameter(np.ones_like(xgrid) * hi)  # [J/kg] stagnation enthalpy
rho: ca.MX = prop_cas_interps["rho"](ca.hcat((ps, hs)).T).T  # [kg/m^3] static density
q: ca.MX = mdot / rho  # [m^3/s] volumetric flowrate
v: ca.MX = q / area  # [m/s] section average flow velocity

#   Isentropic expansion from stagnation to static conditions
solver.subject_to(ps == p - 0.5 * rho * v**2)  # Static pressure
s: ca.MX = prop_cas_interps["s"](ca.hcat((p, h)).T).T  # [J/kg-K] entropy
ss: ca.MX = prop_cas_interps["s"](ca.hcat((ps, hs)).T).T  # [J/kg-K] also entropy
solver.subject_to(s == ss)  # Static entropy == stagnation entropy -> adiabatic process

#   Friction pressure loss
#   delta_p = f * (L/D) * rho * v^2 / 2
#   dp/dx = f * (1/D) * rho * v^2 / 2
mu: ca.MX = prop_cas_interps["mu"](ca.hcat((ps, hs)).T).T  # [Pa-s] viscosity
re: ca.MX = rho * v * dh / mu  # [dimensionless] reynolds number
fx: ca.MX = (
    -(f_bellos(re, roughness) / dh) * rho * v**2 / 2
)  # [Pa/m] friction loss per meter
solver.constrain_derivative(fx, p, x, method="trapezoid")

#   Boundary conditions
solver.subject_to(p >= po)  # No negative pressures, thanks
solver.subject_to(p[0] == pi)  # Boundary conditions
solver.subject_to(p[-1] == po)

# Instrumentation - we don't need these, we just want to see these values at the end
temp: ca.MX = prop_cas_interps["T"](ca.hcat((p, h)).T).T  # [K] stagnation temperature
ts: ca.MX = prop_cas_interps["T"](ca.hcat((ps, hs)).T).T  # [K] static temperature
mach: ca.MX = (
    v / prop_cas_interps["alpha"](ca.hcat((p, h)).T).T
)  # [dimensionless] mach number

# We can solve with no objective purely to satisfy the constraints - this is common for physical systems
# where the behavior is fully defined by conservation laws
sol = solver.solve()

In [ ]:
print(f"mdot: {sol(mdot) * 1000.0:.2f} g/s")

plt.figure(figsize=(8, 2), dpi=200)
plt.title("Pressure [Pa]")
plt.plot(xgrid, sol(ps), color="m", linewidth=3, label="Interior Static")
plt.plot(xgrid, sol(p), color="k", label="Interior Stagnation")
plt.axhline(pi, alpha=0.4, color="r", label="Inlet Stagnation")
plt.axhline(po, alpha=0.4, color="b", label="Outlet Stagnation")
plt.xlabel("x [m]")
plt.legend()

plt.figure(figsize=(8, 2), dpi=200)
plt.title("Velocity [m/s]")
plt.xlabel("x [m]")
plt.plot(xgrid, sol(v), color="k")

plt.figure(figsize=(8, 2))
plt.title("Re")
plt.xlabel("x [m]")
plt.plot(xgrid, sol(re), color="k")

plt.figure(figsize=(8, 2))
plt.title("Density")
plt.xlabel("x [m]")
plt.plot(xgrid, sol(rho), color="k")

plt.figure(figsize=(8, 2))
plt.title("Temperature")
plt.xlabel("x [m]")
plt.plot(xgrid, sol(ts), color="m", linewidth=4, label="static")
plt.plot(xgrid, sol(temp), color="k", label="stagnation")
plt.legend()

plt.figure(figsize=(8, 2))
plt.title("Mach Number")
plt.xlabel("x [m]")
plt.plot(xgrid, sol(mach), color="k")

plt.figure(figsize=(8, 2))
plt.title("p / rho")
plt.xlabel("x [m]")
plt.plot(xgrid, sol(p) / sol(rho), color="k")

plt.figure(figsize=(8, 2))
plt.title("enthalpy")
plt.xlabel("x [m]")
plt.plot(xgrid, sol(hs), color="k", label="static")
plt.plot(xgrid, sol(h), color="b", label="stagnation")
plt.legend()